In [ ]:
'''
    Extraction of frames

    Keep the downloaded video in this same directory as this script
    Provide the filename of the video and the dirname for storing frames
    Ensure that the above directory does not already exists
    You can also specify the sampling rate of frames from the video
'''

import cv2
import os

#Download the required video and place it in the same directory as this and provide the filename

filename = 'Video_1.mp4'
dirname = 'frames1' # filename for storing frames

# Open video using OpenCV
cap = cv2.VideoCapture(filename)

# Set frame count and sampling rate
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
sample_rate = 10  # extract every 10th frame

print("Number of total frames : " ,num_frames) 

#ensure that no directory with this name already exists
os.mkdir(dirname)
for i in range(num_frames):
    ret, frame = cap.read()
    if i % sample_rate == 0:
        cv2.imwrite(f"{dirname}/frame_{i}.jpg", frame)

# # Release video and close all windows
cap.release()
cv2.destroyAllWindows()


Number of total frames :  24022


In [ ]:

'''
  Extraction of feature space from the frames 

  Saves the features in the filename provided file
'''

features_file_name = 'features2.mat'

import torch
import torchvision.transforms as transforms
import PIL.Image
import numpy as np

# Load the pre-trained ResNet model
model = torch.hub.load('pytorch/vision:v0.13.1', 'resnet18', weights="ResNet18_Weights.DEFAULT")

# Define a function to preprocess the image
def preprocess_image(image_path):
  img = PIL.Image.open(image_path)
  img = img.resize((224, 224), PIL.Image.LANCZOS)
  transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])
  img_tensor = transform(img)
  img_tensor = img_tensor.unsqueeze(0)
  return img_tensor

# Define a function to extract features from the image
def extract_features(image_paths, model):
  features_list = []
  for image_path in image_paths:
    img_tensor = preprocess_image(image_path)
    features = model(img_tensor)
    features = features.view(features.size(0), -1) # flatten the feature maps
    features = features.detach().numpy()
    features = np.squeeze(features) # remove batch dimension
    features_list.append(features)
  features_array = np.stack(features_list, axis=0)
  return features_array

# Extract features from an example image and convert to 1D NumPy array
image_paths = []
for i in range(0,num_frames,sample_rate):
  image_paths.append(f"{dirname}/frame_{i}.jpg")
features_array = extract_features(image_paths, model)

# Print the shape of the extracted features
print("Shape of extracted features:", features_array.shape)
# print(features_array)

import scipy.io

scipy.io.savemat(f'{features_file_name}',{'features':features_array})


